In [3]:
import spacy
from spacy.language import Language

In [17]:
model_path = "training/vesteinn/DanskBERT-2023-04-21/model-best"
nlp = spacy.load(model_path)

In [19]:
doc = nlp("Danmarks statsminister hedder Mette Frederiksen fra Socialdemokratiet udtalte i sidste uge følgende:")


/home/kenneth/.local/lib/python3.10/site-packages/spacy/pipeline/entity_linker.py:532: RuntimeWarning: invalid value encountered in divide
  sims = xp.dot(entity_encodings, sentence_encoding_t) / (


# Entities

In [20]:
from spacy import displacy
displacy.render(doc, style="ent")

In [21]:
for ent in doc.ents:
    print(ent.text, ent.label_, ent.kb_id_)

Danmarks LOC NIL
Mette Frederiksen PER Q35
Socialdemokratiet ORG Q12315559
